In [1]:
import pandas as pd
import numpy as np
import pickle 
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Loading Inputs

In [11]:
model = pickle.load(open('../data/06_models/model.pkl/2020-10-09T15.28.01.961Z/model.pkl', 'rb'))
model['pass 1']

In [18]:
model

{'pass 1': <src.utils.modeling.ForecastingModel at 0x7f4e305ce490>,
 'pass 2': <src.utils.modeling.ForecastingModel at 0x7f4de9b28760>,
 'pass 3': <src.utils.modeling.ForecastingModel at 0x7f4de9814a30>,
 'full': <src.utils.modeling.ForecastingModel at 0x7f4de9820a30>}

In [12]:
modeling = {
    'targets': ['DEF0C', 'DE111']
}

In [13]:
cv_splits_positions = pickle.load(open('../data/05_model_input/splits_positions.pkl/2020-10-08T15.33.52.010Z/splits_positions.pkl', 'rb'))
cv_splits_positions

{'pass 1': {'train': slice(0, 542, None), 'val': slice(542, 549, None)},
 'pass 2': {'train': slice(0, 719, None), 'val': slice(719, 726, None)},
 'pass 3': {'train': slice(0, 896, None), 'val': slice(896, 903, None)}}

In [14]:
df_infer = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_infer.hdf', 
    key='df_infer'
)

df_test = capacity_factors_daily_2000to2015 = pd.read_hdf(
    path_or_buf='../data/05_model_input/df_test.hdf', 
    key='df_test'
)

display(df_infer.head(3), df_infer.shape, df_test.head(3), df_test.shape)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2013-01-01,0.219507,0.269458,0.240540,0.263019,0.593148,0.331149,0.555176,0.284140,0.357407,0.410291,...,0.300818,0.363388,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-02,0.083655,0.063571,0.043303,0.005984,0.080731,0.007986,0.066120,0.083709,0.078740,0.108224,...,0.150699,0.133930,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2013-01-03,0.246707,0.229298,0.337413,0.047907,0.279092,0.136004,0.237430,0.320095,0.460150,0.516133,...,0.348737,0.297686,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


(903, 292)

,DE145,DE114,DE146,DE132,DE12A,DE133,DE12C,DE11C,DE118,DE119,...,DEG01,DEG0F,DE275,DE21C,DE234,DE251,DE276,DE278,DE718,DE943
date,,,,,,,,,,,,,,,,,,,,,
2015-06-23,0.366372,0.282144,0.169123,0.102384,0.299887,0.085913,0.264382,0.337191,0.124571,0.252602,...,0.134666,0.162422,0.573021,0.416591,0.663477,0.429502,0.499571,0.522793,0.507254,0.238118
2015-06-24,0.011242,0.008288,0.012873,0.006662,0.006408,0.011105,0.008853,0.012184,0.006574,0.024096,...,0.068389,0.070911,0.063352,0.037593,0.179141,0.085772,0.045303,0.043444,0.097510,0.248150
2015-06-25,0.007218,0.007427,0.004105,0.006482,0.011570,0.007547,0.008719,0.007859,0.003046,0.003604,...,0.002606,0.002760,0.002339,0.003077,0.007730,0.002873,0.003176,0.003211,0.021591,0.055783


(7, 292)

In [15]:
import sys
sys.path.append('..')
scaler = pickle.load(open('../data/05_model_input/scaler.pkl/2020-10-09T13.46.10.640Z/scaler.pkl', 'rb'))

scaler

Pipeline(steps=[('quantiletransformer',
                 QuantileTransformer(output_distribution='normal',
                                     random_state=0)),
                ('makestrictlypositive', MakeStrictlyPositive())])

# Making Predictions

In [16]:
targets = modeling['targets']

In [17]:
preds = {}
for pass_id in cv_splits_positions.keys():
    preds[pass_id] = {}
    for cat in ['train', 'val']:
        yhat = {}
        
        window = df_infer[ cv_splits_positions[pass_id][cat]].index
        start = window[0]
        end = window[-1]
        for district in targets:
            yhat[district] = model[pass_id][district].predict(start, end, scaler)
        preds[pass_id][cat] = pd.DataFrame(yhat)
        
# model trained on entire inference dataset
preds['full'] = {}

for cat in ['train', 'test']:
    
    if cat == 'train':
        df = df_infer
    else: 
        df = df_test
        
    start = df.index[0]
    end = df.index[-1]
        
    preds['full'][cat] = pd.DataFrame(
    {district: model['full'][district].predict(start, end, scaler) for district in targets})

TypeError: 'ForecastingModel' object is not subscriptable

In [9]:
preds['full']['test']

,DEF0C,DE111
2015-06-23,4.595447,5.855856
2015-06-24,4.567227,5.903449
2015-06-25,4.578190,5.802807
2015-06-26,4.671499,5.825727
2015-06-27,4.704972,5.911331
2015-06-28,4.840944,5.943249
2015-06-29,4.725545,5.828873


In [ ]:
gtruth = {}
preds = {}
for pass_id in cv_splits_positions.keys():
    gtruth[pass_id] = {}
    preds[pass_id] = {}
    for cat in ['train', 'val']:
        y = {}
        yhat = {}
        
        window = df_infer[ cv_splits_positions[pass_id][cat] ].index
        start = window[0]
        end = window[-1]
        
        y[district] = df_infer[slice(start, end)][district]
        
        for district in targets:
            yhat[district] = model[pass_id][district].predict(start, end, scaler)
        
        gtruth[pass_id][cat] = pd.DataFrame(y)
        preds[pass_id][cat] = pd.DataFrame(yhat)
        
# model trained on entire inference dataset
gtruth['full'] = {}
preds['full'] = {}

for cat in ['train', 'test']:
    
    if cat == 'train':
        df = df_infer
    else: 
        df = df_test
        
    start = df.index[0]
    end = df.index[-1]
    
    gtruth['full'][cat] = pd.DataFrame(
        {district: df[slice(start, end)][district] for district in targets}
    )
    
    preds['full'][cat] = pd.DataFrame(
        {district: model['full'][district].predict(start, end, scaler) for district in targets}
    )

In [ ]:
preds['full']

In [ ]:
gtruth['full']

# Evaluating

In [ ]:
mean_squared_error(y, y_hat, multioutput='raw_values') ** 0.5